In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pyperclip
import time


## getting started 




In [2]:
## personal info

from personal import chrome_driver_path, naver_url, craw_url, naver_id, naver_pw

'''personal.py code look like this, fill the variable 
chrome_driver_path = "/chromedriver_win32" 
naver_url = 
craw_url = 
naver_id = 
naver_pw = 
'''


'personal.py code look like this, fill the variable \nchrome_driver_path = "/chromedriver_win32" \nnaver_url = \ncraw_url = \nnaver_id = \nnaver_pw = \n'

In [ ]:
## 기본 로그인 basic login
## cccv 로 안막히는 네이버님 감사합니다. 

driver = webdriver.Chrome(chrome_driver_path)
driver.get(naver_url)


login_button = driver.find_element(By.CSS_SELECTOR, "#account > a")
login_button.click()

input_id = driver.find_element(By.CSS_SELECTOR, "#id")
input_pw = driver.find_element(By.CSS_SELECTOR, "#pw")

##input_id.send_keys(naver_id)
##input_pw.send_keys(naver_pw)

input_id.click()
pyperclip.copy(naver_id)
input_id.send_keys(Keys.CONTROL, 'v')
time.sleep(1)

input_pw.click()
pyperclip.copy(naver_pw)
input_pw.send_keys(Keys.CONTROL, 'v')
time.sleep(1)

#browser.execute_script(f"document.getElementsByName('pw')[0].value='{naver_pw}'")

login_send = driver.find_element(By.CSS_SELECTOR, "#log\.login")
login_send.click()

In [5]:
## 게시판 접근 enter board 

driver.get(craw_url)
view_btn = driver.find_element(By.CSS_SELECTOR,"#menuLink385")
view_btn.click()

In [6]:
## 검색 search 

## 먼저, 네이버 카페의 경우 전체 페이지가 iframe 이므로, iframe > frame 으로 변환하여 내부 CSS 요소를 탐색 
## first, in naver cafe case.entire pages is iframe, convert iframe > frame to find internel CSS elements 

search_blank = "#query"
excute_btn = "#main-area > div.list-search > form > div.input_search_area > button"
item_search = "3060"
mode_sel = "#searchOptionSelectDiv > a"
change_view = "#searchOptionSelectDiv > ul > li:nth-child(1) > a"

i_search = driver.find_element(By.CSS_SELECTOR, "#cafe_main")

driver.switch_to.frame(i_search)
time.sleep(1)





#topLayerQueryInput
#main-area > div.list-search > form > div.input_search_area >
#main-area > div.list-search > form > div.input_search_area > div > query

In [7]:
## inline frame element click 
search = driver.find_element(By.CSS_SELECTOR,search_blank)
time.sleep(1)

search.click()
pyperclip.copy(item_search)
search.send_keys(Keys.CONTROL, 'v')
time.sleep(1)

excute = driver.find_element(By.CSS_SELECTOR,excute_btn)
excute.click()